# Predict user car sale price

This version of the analysis uses a deep learning model with fewer number of features - i.e. only the features that were identified as meaningful in V3 of the analysis based on regression. 

# 1. Setup

In [6]:
%pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 857.5 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 14.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 13.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 610.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.7/123.7 kB 1.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 18.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 5.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 2.9 MB/s eta 0

In [4]:
%pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
# imports
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split 

import pickle

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


2022-06-25 01:18:26.281586: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-25 01:18:26.282059: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.9.1


In [8]:
# Make NumPy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

## load model data

In [9]:
import pickle
# load car price data set 
with open('./df_model_minimal.pkl', 'rb') as handle:
    df = pickle.load(handle)
df.shape

(33956, 22)

In [10]:
# examine
df.head(2)

,gas,100k-150k,150k-200k,fwd,automatic,sedan,200k-250k,fair,honda,other_type,...,other_fuel,0-010k,diesel,4wd,25k-50k,10k-25k,other_transmission,pickup,year,price
21171,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,1,0,2018,16990.0
21174,1,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,2003,11975.0


In [11]:
# convert to a numpy matrix
dataset = df.values
dataset[:3]


array([[    1.,     0.,     0.,     1.,     0.,     1.,     0.,     0.,
            0.,     0.,     0.,     1.,     0.,     0.,     0.,     0.,
            0.,     1.,     1.,     0.,  2018., 16990.],
       [    1.,     0.,     0.,     0.,     1.,     0.,     1.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     1.,  2003., 11975.],
       [    0.,     0.,     0.,     0.,     1.,     0.,     1.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     1.,     1.,
            0.,     0.,     0.,     0.,  2003., 21950.]])

In [16]:
# split into input (X) and output (Y) variables
X = dataset[:,1:21]
y = dataset[:,21]

In [17]:
print(X)

[[   0.    0.    1. ...    1.    0. 2018.]
 [   0.    0.    0. ...    0.    1. 2003.]
 [   0.    0.    0. ...    0.    0. 2003.]
 ...
 [   0.    0.    1. ...    0.    0. 2017.]
 [   1.    0.    1. ...    0.    0. 2015.]
 [   0.    0.    0. ...    0.    0. 2011.]]


In [18]:
print(y)

[16990. 11975. 21950. ... 31990. 14590. 32990.]


In [15]:
print(f"test sample size: {len(y)}")

test sample size: 33956


In [19]:
# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, 
                                                    random_state = 42) 

### Examine data set

In [21]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
gas,33956.0,0.839793,0.366804,0.0,1.0,1.0,1.0,1.0
100k-150k,33956.0,0.241636,0.428081,0.0,0.0,0.0,0.0,1.0
150k-200k,33956.0,0.104753,0.306240,0.0,0.0,0.0,0.0,1.0
fwd,33956.0,0.570474,0.495016,0.0,0.0,1.0,1.0,1.0
automatic,33956.0,0.758923,0.427743,0.0,1.0,1.0,1.0,1.0
sedan,33956.0,0.303275,0.459680,0.0,0.0,0.0,1.0,1.0
200k-250k,33956.0,0.031305,0.174144,0.0,0.0,0.0,0.0,1.0
fair,33956.0,0.014548,0.119737,0.0,0.0,0.0,0.0,1.0
honda,33956.0,0.066763,0.249615,0.0,0.0,0.0,0.0,1.0
other_type,33956.0,0.069472,0.254259,0.0,0.0,0.0,0.0,1.0


In [22]:
df.describe().transpose()[['mean', 'std']]

,mean,std
gas,0.839793,0.366804
100k-150k,0.241636,0.428081
150k-200k,0.104753,0.306240
fwd,0.570474,0.495016
automatic,0.758923,0.427743
sedan,0.303275,0.459680
200k-250k,0.031305,0.174144
fair,0.014548,0.119737
honda,0.066763,0.249615
other_type,0.069472,0.254259


# 2. Define Baseline Neural Network Model

In [27]:
# define base model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(20, 
                    input_dim=20, 
                    kernel_initializer='normal', 
                    activation='relu'))
#     model.add(Dense(32, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, 
                    kernel_initializer='normal'))

    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [28]:
# compile the network
estimator = KerasRegressor(build_fn=baseline_model,
                           epochs=100, 
                           batch_size=5, 
                           verbose=0)

/tmp/ipykernel_19/3813662269.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasRegressor(build_fn=baseline_model,


In [29]:
# add crossvalidation
kfold = KFold(n_splits=10)

In [ ]:
# start 6:22pm pacific
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

In [30]:
# fit and evaluate model
results = cross_val_score(estimator, X_train, y_train, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: -88861888.00 (3160648.24) MSE


In [31]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 02:50:49
